<a href="https://colab.research.google.com/github/sakuna47/Stock_Price_Prediction-/blob/Frontend/SPP_Frontend_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pickle
import numpy as np
import yfinance as yf

# Load the model and scaler
with open("stock_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Streamlit app
st.title("Stock Price Prediction Web App")

# Input field for stock ticker
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL):")

# Predict button
if st.button("Predict"):
    if ticker:
        # Download historical stock data
        data = yf.download(ticker, start="2010-01-01", end="2023-01-01")
        if not data.empty:
            # Preprocess the data
            data = data.fillna(method="ffill")
            X = data.drop(["Close"], axis=1)
            X_scaled = scaler.transform(X)
            # Make prediction
            prediction = model.predict(X_scaled)
            st.success(f"Predicted Closing Price: {prediction[-1]:.2f}")
        else:
            st.error("Invalid stock ticker or no data available.")
    else:
        st.warning("Please enter a stock ticker.")
